Change Log:
- v32 - remove building with too large percent error
- v20 - lower learning rate = 0.0008, add height_r and top_floor index
- v16 - change parameters (some from model 05), rm reduce to 68
- v14 - change random seed
- v10 - remove features based on feature importance results (exp-l04-per-area)

### Import packages

In [1]:
import time
import pickle
import itertools
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from utilities import *

In [2]:
model_name = 'model-35-lgb-remove_outlier_03-2'

### Params

In [3]:
is_log_target = True
is_per_area_target = True
rm_outlier = True
outlier_pct = 3 # 1, 3, 5

map_target = {
    (True, True):   'log_per_area_total_price',
    (True, False):  'log_total_price',
    (False, True):  'per_area_total_price',
    (False, False): 'total_price'
}
col_target = map_target[(is_log_target, is_per_area_target)]

#cols_feats_add = ['encoded_latlon', 'building_per_land_area'] # add feats into model
cols_num_add = ['building_per_land_area', 'height_r']
cols_cat_add = ['encoded_latlon']

#cols_feats_rm = [] # TODO: implement before using it
#['day_in_year_txn_dt', 'day_in_year_building_complete_dt', 'dt_diff_building_complete_dt_txn_dt'] # dt num feats
#['dow_txn_dt', 'dow_building_complete_dt', 'month_txn_dt', 'month_building_complete_dt',
# 'quarter_txn_dt', 'quarter_building_complete_dt', 'year_txn_dt', 'year_building_complete_dt'] # dt cat feats

### Processing

In [4]:
def pre_processing(df, cols_num, cols_cat): #, cols_feats
    # Convert types
    df[cols_num] = df[cols_num].astype('float32')
    
    # generate target
    if 'total_price' in df:
        df['total_price'] = df['total_price'].astype('float32')
        df['log_total_price'] = np.log1p(df['total_price'])
        df['per_area_total_price'] = df['total_price'] / df['building_area']
        df['log_per_area_total_price'] = np.log1p(df['per_area_total_price'])
        df['target'] = df[col_target]

    # generate lat/lon encoded
    kmeans = pickle.load(open('./object_kmeans_k6_combined.pkl', 'rb'))
    df['encoded_latlon'] = kmeans.predict(df[['lat','lon']])
    
    # generate dt feats
    df = create_dt_feats(df, 'txn_dt')
    df = create_dt_feats(df, 'building_complete_dt')
    df['dt_diff_building_complete_dt_txn_dt'] = df['txn_dt'] - df['building_complete_dt']
    
    # generate building to land ratio
    df['building_per_land_area'] = df['building_area'] / df['land_area']
 
    #cols_feats = cols_num + cols_cat + cols_feats_add
    
    # fix town and city
    df['town'] = df['city'].astype('str')+'-'+df['town'].astype('str')
    df['village'] = df['town'].astype('str')+'-'+df['village'].astype('str')
    
    # height ratio
    df['height_r'] = df['txn_floor'] / df['total_floor']
    df['top_floor'] = df['txn_floor'] == df['total_floor']
    
    return df  #, cols_feats

In [5]:
from feature_engineering import CategoricalColumnsEncoder
from feature_engineering import TargetMeanEncoding
from feature_engineering import MultiLabelEncoding

class FeatureEngineering():
    def __init__(self, cols_num, cols_cat, cols_feats, col_target, n_ran_encode=4):
        # cols
        self.cols_num = cols_num
        self.cols_cat = cols_cat
        self.cols_feats = cols_feats
        self.col_target = col_target
        
        # Numbers
        self.n_ran_encode = n_ran_encode
        
        # encoders
        #self.tar_encoder = TargetMeanEncoding(col_target = self.col_target)
        #self.ran_encoder = MultiLabelEncoding(self.n_ran_encode)
        self.cat_astyper = CategoricalColumnsEncoder(mode='pandas')
        
    def fit_transform(self, df):
        df = df.copy()
    
#        encoded1 = self.tar_encoder.fit_transform(df[self.cols_cat], df[self.col_target])
#        encoded2 = self.ran_encoder.fit_transform(df[self.cols_cat])
        self.cat_astyper.fit_transform(df, self.cols_cat)    
        return df
#        return pd.concat([df, encoded1, encoded2], axis=1)
    
    def transform(self, df):
        df = df.copy()
#        encoded1 = self.tar_encoder.transform(df[self.cols_cat])
#        encoded2 = self.ran_encoder.transform(df[self.cols_cat])     # TODO: add generated feats into cols_feats
        self.cat_astyper.transform(df)
        return df
#        return pd.concat([df, encoded1, encoded2], axis=1)


In [6]:
def post_processing(y_pred, df):
    """Args: 
          y_pred (1d np.array): raw prediction y
          df (pd.Dataframe): input feature df
       Return: np.array
    """
    y_pred_final = y_pred.copy()
    
    if is_log_target:
        y_pred_final = np.expm1(y_pred_final)
        y_pred_final = np.clip(y_pred_final, 0, None)

    if is_per_area_target:
        y_pred_final = y_pred_final * df['building_area'].values
        
    return y_pred_final

### Columns

In [7]:
cols_num = ['txn_dt', 'total_floor', 'building_complete_dt', 'parking_area', 'parking_price', 'txn_floor',
            'land_area', 'building_area', 'lat', 'lon', 'village_income_median', 'town_population', 'town_area',
            'town_population_density', 'doc_rate', 'master_rate', 'bachelor_rate', 'jobschool_rate',
            'highschool_rate', 'junior_rate', 'elementary_rate', 'born_rate', 'death_rate', 'marriage_rate',
            'divorce_rate', 'N_50', 'N_500', 'N_1000', 'N_5000', 'N_10000', 'I_10', 'I_50', 'I_100', 'I_250',
            'I_500', 'I_1000', 'I_5000', 'I_10000', 'I_MIN', 'II_10', 'II_50', 'II_100', 'II_250', 'II_500',
            'II_1000', 'II_5000', 'II_10000', 'II_MIN', 'III_10', 'III_50', 'III_100', 'III_250', 'III_500',
            'III_1000', 'III_5000', 'III_10000', 'III_MIN', 'IV_10', 'IV_50', 'IV_100', 'IV_250', 'IV_500',
            'IV_1000', 'IV_5000', 'IV_10000', 'IV_MIN', 'V_10', 'V_50', 'V_100', 'V_250', 'V_500', 'V_1000',
            'V_5000', 'V_10000', 'V_MIN', 'VI_50', 'VI_100', 'VI_250', 'VI_500', 'VI_1000', 'VI_5000', 'VI_10000',
            'VI_MIN', 'VII_10', 'VII_50', 'VII_100', 'VII_250', 'VII_500', 'VII_1000', 'VII_5000', 'VII_10000',
            'VII_MIN', 'VIII_10', 'VIII_50', 'VIII_100', 'VIII_250', 'VIII_500', 'VIII_1000', 'VIII_5000',
            'VIII_10000', 'VIII_MIN', 'IX_10', 'IX_50', 'IX_100', 'IX_250', 'IX_500', 'IX_1000', 'IX_5000',
            'IX_10000', 'IX_MIN', 'X_10', 'X_50', 'X_100', 'X_250', 'X_500', 'X_1000', 'X_5000', 'X_10000', 'X_MIN',
            'XI_10', 'XI_50', 'XI_100', 'XI_250', 'XI_500', 'XI_1000', 'XI_5000', 'XI_10000', 'XI_MIN', 'XII_10',
            'XII_50', 'XII_100', 'XII_250', 'XII_500', 'XII_1000', 'XII_5000', 'XII_10000', 'XII_MIN', 'XIII_10',
            'XIII_50', 'XIII_100', 'XIII_250', 'XIII_500', 'XIII_1000', 'XIII_5000', 'XIII_10000', 'XIII_MIN',
            'XIV_10', 'XIV_50', 'XIV_100', 'XIV_250', 'XIV_500', 'XIV_1000', 'XIV_5000', 'XIV_10000', 'XIV_MIN']
cols_cat = ['building_material', 'city', 'building_type', 'building_use', 'parking_way', 'town', 'village',
            'I_index_50', 'I_index_500', 'I_index_1000', 'II_index_50', 'II_index_500', 'II_index_1000',
            'III_index_50', 'III_index_500', 'III_index_1000', 'IV_index_50', 'IV_index_500', 'IV_index_1000',
            'IV_index_5000', 'V_index_50', 'V_index_500', 'V_index_1000', 'VI_10', 'VI_index_50', 'VI_index_500',
            'VI_index_1000', 'VII_index_50', 'VII_index_500', 'VII_index_1000', 'VIII_index_50', 'VIII_index_500',
            'VIII_index_1000', 'IX_index_50', 'IX_index_500', 'IX_index_1000', 'IX_index_5000', 'X_index_50',
            'X_index_500', 'X_index_1000', 'XI_index_50', 'XI_index_500', 'XI_index_1000', 'XI_index_5000',
            'XI_index_10000', 'XII_index_50', 'XII_index_500', 'XII_index_1000', 'XIII_index_50', 'XIII_index_500',
            'XIII_index_1000', 'XIII_index_5000', 'XIII_index_10000', 'XIV_index_50', 'XIV_index_500',
            'XIV_index_1000']
cols_feats = cols_num + cols_cat

# informational cols
cols_dt = ['txn_dt', 'building_complete_dt']
cols_bin = ['I_index_50', 'I_index_500', 'I_index_1000', 'II_index_50', 'II_index_500', 'II_index_1000',
            'III_index_50', 'III_index_500', 'III_index_1000', 'IV_index_50', 'IV_index_500', 'IV_index_1000',
            'IV_index_5000', 'V_index_50', 'V_index_500', 'V_index_1000', 'VI_10', 'VI_index_50', 'VI_index_500',
            'VI_index_1000', 'VII_index_50', 'VII_index_500', 'VII_index_1000', 'VIII_index_50', 'VIII_index_500',
            'VIII_index_1000', 'IX_index_50', 'IX_index_500', 'IX_index_1000', 'IX_index_5000', 'X_index_50',
            'X_index_500', 'X_index_1000', 'XI_index_50', 'XI_index_500', 'XI_index_1000', 'XI_index_5000',
            'XI_index_10000', 'XII_index_50', 'XII_index_500', 'XII_index_1000', 'XIII_index_50', 'XIII_index_500',
            'XIII_index_1000', 'XIII_index_5000', 'XIII_index_10000', 'XIV_index_50', 'XIV_index_500',
            'XIV_index_1000']

### Read data

In [8]:
train = pd.read_csv('dataset/train.csv', dtype=object)
test = pd.read_csv('dataset/test.csv', dtype=object)
display(train.head(5))
display(test.head(5))

id_cv = []
for i in range(5):
    id_one_fold = pd.read_csv(f'dataset/id_cv{i}.csv', dtype=object)['building_id']
    id_cv.append(id_one_fold)

,building_id,building_material,city,txn_dt,total_floor,building_type,building_use,building_complete_dt,parking_way,parking_area,...,XIV_500,XIV_index_500,XIV_1000,XIV_index_1000,XIV_5000,XIV_index_5000,XIV_10000,XIV_index_10000,XIV_MIN,total_price
0,e3mMIMR3JJqCaXz1,8,21,18674,4,3,2,6271,2,NaN,...,58,1,157,1,2483,1,6011,1,34.469802848528126,647603.7704719775
1,LgwzgklNvy4QCtq5,8,7,18800,5,1,2,7885,2,NaN,...,28,1,115,1,15872,1,32221,1,40.07357348973039,3321452.0
2,ucIR2NLLsC3T650L,8,7,19289,4,1,2,6028,2,NaN,...,78,1,212,1,15760,1,32228,1,54.46208108983661,9570885.316206373
3,jre1pJhcQj91Kdky,8,21,20385,24,0,2,18325,0,NaN,...,20,1,125,1,2568,1,7271,1,99.62896580763416,14215011.127100045
4,rQpYpY9nRG7X5mmr,1,21,20657,2,4,2,6880,2,NaN,...,18,1,47,1,2587,1,7442,1,124.1312362968124,762711.9757135696


,building_id,building_material,city,txn_dt,total_floor,building_type,building_use,building_complete_dt,parking_way,parking_area,...,XIV_250,XIV_500,XIV_index_500,XIV_1000,XIV_index_1000,XIV_5000,XIV_index_5000,XIV_10000,XIV_index_10000,XIV_MIN
0,X5gsdTWGS3W7JJQB,1,13,18634,2,4,2,1857,2,NaN,...,7,21,1,129,1,3991,1,8409,1,105.4622312795739
1,BTshNOJyKHnT2YIT,8,7,20199,10,3,2,16011,2,NaN,...,18,44,1,67,1,1950,1,10725,1,100.81980868401074
2,dhdymr0lV8N5kZOT,8,12,19769,4,4,10,18294,2,NaN,...,0,3,1,35,1,3707,1,19459,1,259.8005430183116
3,VEwyGGMcD56w5BOc,10,7,20479,19,2,0,17837,2,NaN,...,16,60,1,144,1,6487,1,29400,1,22.941906168328487
4,wmUeMoJZfsqaSX9b,8,3,18164,13,0,2,13272,2,NaN,...,4,18,1,76,1,1346,1,3280,1,181.21309507601745


In [9]:
# Only train on data in this list
if rm_outlier:
    training_buildings = pd.read_csv('dataset/building_we_care_%d.csv'%outlier_pct)
    print('N outliers removed:', len(train) - len(training_buildings))

N outliers removed: 1800


### Preprocessing

In [10]:
train = pre_processing(train, cols_num, cols_cat)
test = pre_processing(test, cols_num, cols_cat)

/home/skyhuang/.local/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator KMeans from version 0.20.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/skyhuang/.local/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator KMeans from version 0.20.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [11]:
cols_num = cols_num + [c for c in cols_num_add if c not in cols_num]
cols_cat = cols_cat + [c for c in cols_cat_add if c not in cols_cat]
cols_feats = cols_num + cols_cat

### Remove feats based on shuffle results

In [12]:
# PARS #
N_feats_rm = 70
# PARS #

from vars_03 import score_base_per_area, shuffle_results_per_area

feat_imp = pd.DataFrame({'feats': shuffle_results_per_area[:,0],
                         'avg':shuffle_results_per_area[:,1]})
feat_imp['diff'] = feat_imp['avg'] - score_base_per_area
feat_imp = feat_imp.sort_values(by='diff', ascending=False)
print(feat_imp.reset_index().values)

feats_rm = list(feat_imp.iloc[:(N_feats_rm+1)]['feats'].values)
feats_rm.remove('none')
print(len(feats_rm))
print(feats_rm)

print('N feats (before):', len(cols_feats))
cols_feats = list(set(cols_feats) - set(feats_rm))
print('N feats (after):', len(cols_feats))
print(cols_feats)

[[0 'divorce_rate' 5709.20035 3.333296666667593]
 [1 'I_250' 5709.200316666666 3.333263333333889]
 [2 'XI_100' 5708.200346666667 2.33329333333495]
 [3 'VI_index_500' 5707.867050000001 1.9999966666682667]
 [4 'IX_index_50' 5707.86704 1.9999866666676098]
 [5 'VI_500' 5707.866983333333 1.9999300000008589]
 [6 'XIV_10' 5707.533710000001 1.666656666668132]
 [7 'XI_index_500' 5707.200386666666 1.3333333333339397]
 [8 'VI_index_1000' 5707.200383333334 1.3333300000012969]
 [9 'V_100' 5707.200373333333 1.33332000000064]
 [10 'VII_10' 5707.200363333334 1.333310000001802]
 [11 'X_100' 5707.200336666666 1.3332833333333838]
 [12 'XII_index_50' 5706.867056666667 1.0000033333344618]
 [15 'X_index_500' 5706.867050000001 0.9999966666682667]
 [14 'X_10' 5706.867050000001 0.9999966666682667]
 [13 'II_10' 5706.867050000001 0.9999966666682667]
 [16 'II_index_500' 5706.867046666666 0.9999933333338049]
 [17 'V_index_1000' 5706.867046666666 0.9999933333338049]
 [18 'XIII_10' 5706.867043333335 0.99999000000207

### Params

In [13]:
# params
# parameters from model 10: 24, 300, 0.5, 10, 0.1, 0.0
# parameters from model 5: 24, 258, 0.4, 6, 0.04, 0.02
params = {'task': 'train',
          'boosting_type': 'gbdt',
          'objective': 'mse',
          'metric': 'mape',
          'learning_rate': 0.001, # 0.001
          'max_depth': 24,
          'num_leaves': 279,           # may reduce in dim-reduction exp
          'feature_fraction': 0.45,
          'min_data_in_leaf': 8,
          'lambda_l1': 0.07,
          'lambda_l2': 0.01,
          'seed': 666,
          'bagging_freq': 10,
          'bagging_fraction': 0.9, 
         }

lgb_other_params = {'num_boost_round': 1000000,  # expected to stop 250000
                    'verbose_eval': 2000,
                    'early_stopping_rounds': 1000,}


### kfold cross-predict

In [14]:
t0 = time.time()

predicts_val = []
predicts_test = test[['building_id']].copy()
best_iterations = []
scores = []
for i, id_fold in enumerate(id_cv):
    #id_fold = id_fold['building_id']
    # split train, val
    df_train = train.loc[~train['building_id'].isin(id_fold)].copy()
    df_val = train.loc[train['building_id'].isin(id_fold)].copy()
    df_test = test.copy()
    
    # feat eng
    feat_eng = FeatureEngineering(cols_num, cols_cat, cols_feats, col_target)
    df_train = feat_eng.fit_transform(df_train)
    df_val = feat_eng.transform(df_val)
    df_test = feat_eng.transform(df_test)

    if rm_outlier:
        df_train = df_train.merge(training_buildings, on='building_id', how='inner').reset_index()

    # Construct lgb dataset
    lgb_train = lgb.Dataset(df_train[cols_feats], df_train[col_target]).construct()
    lgb_val = lgb.Dataset(df_val[cols_feats], df_val[col_target], reference=lgb_train).construct()
        
    # train model    
    model = lgb.train(params, lgb_train, valid_sets=lgb_val, **lgb_other_params, keep_training_booster=True)
    best_iterations.append(model.best_iteration)

    # Predict on val set
    y_pred = model.predict(df_val[cols_feats])
    mse = mean_squared_error(df_val[col_target], y_pred)
    mae = mean_absolute_error(df_val[col_target], y_pred)
    mape = cal_mape(df_val[col_target], y_pred)
    
    y_pred_final = post_processing(y_pred, df_val)
    score = cal_score(df_val['total_price'], y_pred_final)
    scores.append(score)

    print('(Fold %d) score, mse, mae, mape: %.5f, %.5f, %.5f, %.5f.' % (i, score, mse, mae, mape))
    predicts_val.append(pd.DataFrame({'building_id': df_val['building_id'],
                                      'total_price': df_val['total_price'],
                                      'total_price_predict': y_pred_final}))

    # Predict on test set
    y_pred_test = model.predict(df_test[cols_feats])
    predicts_test['target_{}'.format(i)] = y_pred_test

Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's mape: 0.0156812
[4000]	valid_0's mape: 0.0113693
[6000]	valid_0's mape: 0.010446
[8000]	valid_0's mape: 0.0100298
[10000]	valid_0's mape: 0.00978329
[12000]	valid_0's mape: 0.00962048
[14000]	valid_0's mape: 0.00950517
[16000]	valid_0's mape: 0.00942001
[18000]	valid_0's mape: 0.00935428
[20000]	valid_0's mape: 0.00930161
[22000]	valid_0's mape: 0.00925879
[24000]	valid_0's mape: 0.00922314
[26000]	valid_0's mape: 0.00919302
[28000]	valid_0's mape: 0.00916689
[30000]	valid_0's mape: 0.00914461
[32000]	valid_0's mape: 0.0091255
[34000]	valid_0's mape: 0.00910859
[36000]	valid_0's mape: 0.00909381
[38000]	valid_0's mape: 0.00908066
[40000]	valid_0's mape: 0.00906917
[42000]	valid_0's mape: 0.00905862
[44000]	valid_0's mape: 0.00904919
[46000]	valid_0's mape: 0.00904073
[48000]	valid_0's mape: 0.00903292
[50000]	valid_0's mape: 0.00902601
[52000]	valid_0's mape: 0.00901962
[54000]	valid_0's mape: 0.00901378
[

[78000]	valid_0's mape: 0.00900239
[80000]	valid_0's mape: 0.00900026
[82000]	valid_0's mape: 0.00899823
[84000]	valid_0's mape: 0.00899637
[86000]	valid_0's mape: 0.00899468
[88000]	valid_0's mape: 0.00899296
[90000]	valid_0's mape: 0.00899141
[92000]	valid_0's mape: 0.00898996
[94000]	valid_0's mape: 0.00898853
[96000]	valid_0's mape: 0.00898718
[98000]	valid_0's mape: 0.00898592
[100000]	valid_0's mape: 0.00898475
[102000]	valid_0's mape: 0.00898371
[104000]	valid_0's mape: 0.00898269
[106000]	valid_0's mape: 0.00898171
[108000]	valid_0's mape: 0.0089808
[110000]	valid_0's mape: 0.00897996
[112000]	valid_0's mape: 0.00897918
[114000]	valid_0's mape: 0.00897837
[116000]	valid_0's mape: 0.00897757
[118000]	valid_0's mape: 0.00897685
[120000]	valid_0's mape: 0.00897616
[122000]	valid_0's mape: 0.00897555
[124000]	valid_0's mape: 0.00897492
[126000]	valid_0's mape: 0.00897438
[128000]	valid_0's mape: 0.00897384
[130000]	valid_0's mape: 0.0089733
[132000]	valid_0's mape: 0.00897277
[1340

[188000]	valid_0's mape: 0.00898864
[190000]	valid_0's mape: 0.00898848
[192000]	valid_0's mape: 0.00898827
[194000]	valid_0's mape: 0.00898808
[196000]	valid_0's mape: 0.0089879
[198000]	valid_0's mape: 0.00898775
[200000]	valid_0's mape: 0.00898758
[202000]	valid_0's mape: 0.00898743
[204000]	valid_0's mape: 0.00898729
[206000]	valid_0's mape: 0.00898713
[208000]	valid_0's mape: 0.00898701
[210000]	valid_0's mape: 0.00898687
[212000]	valid_0's mape: 0.00898674
[214000]	valid_0's mape: 0.00898661
[216000]	valid_0's mape: 0.0089865
[218000]	valid_0's mape: 0.00898639
[220000]	valid_0's mape: 0.00898625
[222000]	valid_0's mape: 0.00898615
[224000]	valid_0's mape: 0.00898603
[226000]	valid_0's mape: 0.00898593
[228000]	valid_0's mape: 0.00898583
[230000]	valid_0's mape: 0.00898572
[232000]	valid_0's mape: 0.00898563
[234000]	valid_0's mape: 0.0089855
[236000]	valid_0's mape: 0.0089854
[238000]	valid_0's mape: 0.00898532
[240000]	valid_0's mape: 0.00898525
[242000]	valid_0's mape: 0.00898

[210000]	valid_0's mape: 0.00903591
[212000]	valid_0's mape: 0.00903573
[214000]	valid_0's mape: 0.00903558
[216000]	valid_0's mape: 0.00903539
[218000]	valid_0's mape: 0.00903525
[220000]	valid_0's mape: 0.0090351
[222000]	valid_0's mape: 0.00903494
[224000]	valid_0's mape: 0.0090348
[226000]	valid_0's mape: 0.00903466
[228000]	valid_0's mape: 0.00903453
[230000]	valid_0's mape: 0.00903434
[232000]	valid_0's mape: 0.00903423
[234000]	valid_0's mape: 0.00903412
[236000]	valid_0's mape: 0.00903397
[238000]	valid_0's mape: 0.00903388
[240000]	valid_0's mape: 0.00903374
[242000]	valid_0's mape: 0.00903364
[244000]	valid_0's mape: 0.0090335
[246000]	valid_0's mape: 0.0090334
[248000]	valid_0's mape: 0.00903328
[250000]	valid_0's mape: 0.00903318
[252000]	valid_0's mape: 0.00903308
[254000]	valid_0's mape: 0.00903298
[256000]	valid_0's mape: 0.00903286
[258000]	valid_0's mape: 0.00903275
[260000]	valid_0's mape: 0.00903265
[262000]	valid_0's mape: 0.00903256
[264000]	valid_0's mape: 0.00903

[228000]	valid_0's mape: 0.00895622
[230000]	valid_0's mape: 0.00895615
[232000]	valid_0's mape: 0.00895605
[234000]	valid_0's mape: 0.00895595
[236000]	valid_0's mape: 0.00895586
[238000]	valid_0's mape: 0.00895577
[240000]	valid_0's mape: 0.00895569
[242000]	valid_0's mape: 0.00895562
[244000]	valid_0's mape: 0.00895554
[246000]	valid_0's mape: 0.00895544
[248000]	valid_0's mape: 0.00895537
[250000]	valid_0's mape: 0.00895529
[252000]	valid_0's mape: 0.00895523
[254000]	valid_0's mape: 0.00895518
[256000]	valid_0's mape: 0.00895512
[258000]	valid_0's mape: 0.00895504
[260000]	valid_0's mape: 0.00895497
[262000]	valid_0's mape: 0.00895492
[264000]	valid_0's mape: 0.00895484
[266000]	valid_0's mape: 0.00895479
[268000]	valid_0's mape: 0.00895474
[270000]	valid_0's mape: 0.0089547
[272000]	valid_0's mape: 0.00895463
[274000]	valid_0's mape: 0.00895457
[276000]	valid_0's mape: 0.00895453
[278000]	valid_0's mape: 0.00895447
[280000]	valid_0's mape: 0.00895443
[282000]	valid_0's mape: 0.00

48s per 2000 - ETA (all including one): 16.6 hrs

In [15]:
print('Total run time:', time.time() - t0)

Total run time: 48665.24586009979


In [16]:
print('Kfold cv score (mean, stdev, all):', np.mean(scores), np.std(scores), scores) # 5868.87314

Kfold cv score (mean, stdev): 5983.276109811884 37.45122433871052


In [17]:
predicts_test.head()

,building_id,target_0,target_1,target_2,target_3,target_4
0,X5gsdTWGS3W7JJQB,15.108238,15.144833,15.133459,15.191798,15.078641
1,BTshNOJyKHnT2YIT,13.137452,13.111892,13.136005,13.137316,13.131938
2,dhdymr0lV8N5kZOT,13.644638,13.660838,13.668330,13.652730,13.660801
3,VEwyGGMcD56w5BOc,14.824193,14.819720,14.783903,14.825478,14.784292
4,wmUeMoJZfsqaSX9b,12.152215,12.126819,12.069087,12.180481,12.175286


In [18]:
pd.concat(predicts_val, axis=0).to_csv('output/'+model_name+'-cv.csv', index=False)

predicts_test['target_mean'] = \
    predicts_test[['target_'+str(i) for i in range(len(id_cv))]].mean(axis=1)
predicts_test['total_price'] = post_processing(predicts_test['target_mean'].values, df_test)
predicts_test[['building_id', 'total_price']].to_csv('output/'+model_name+'-test-kfold.csv', index=False)

In [19]:
print('Best iterations:', best_iterations)
print('Mean best iterations:', int(np.mean(best_iterations)))

Best iterations: [372876, 339811, 428481, 429000, 366200]
Mean best iterations: 387273


### One-fold test

In [20]:
lgb_other_params_one = {'num_boost_round': int(np.mean(best_iterations)*1.2),
                        'verbose_eval': 2000,}
lgb_other_params_one

{'num_boost_round': 464728, 'verbose_eval': 2000}

In [21]:
params

{'task': 'train',
 'boosting_type': 'gbdt',
 'objective': 'mse',
 'metric': 'mape',
 'learning_rate': 0.001,
 'max_depth': 24,
 'num_leaves': 279,
 'feature_fraction': 0.45,
 'min_data_in_leaf': 8,
 'lambda_l1': 0.07,
 'lambda_l2': 0.01,
 'seed': 666,
 'bagging_freq': 10,
 'bagging_fraction': 0.9}

In [22]:
t10 = time.time()

df_train = train.copy()
df_test = test.copy()

# feat eng
feat_eng = FeatureEngineering(cols_num, cols_cat, cols_feats, col_target)
df_train = feat_eng.fit_transform(df_train)
df_test = feat_eng.transform(df_test)

if rm_outlier:
    df_train = df_train.merge(training_buildings, on='building_id', how='inner').reset_index()

# Construct lgb dataset
lgb_train = lgb.Dataset(df_train[cols_feats], df_train[col_target]).construct()

# train model    
model = lgb.train(params, lgb_train, valid_sets=lgb_train, **lgb_other_params_one, keep_training_booster=True)

y_pred = model.predict(df_test[cols_feats])
y_pred_final = post_processing(y_pred, df_test)

df_test_pred = pd.DataFrame({'building_id': df_test['building_id'], 
                             'total_price': y_pred_final})

t11 = time.time()

display(df_test_pred.head(20))

[2000]	training's mape: 0.0135386
[4000]	training's mape: 0.00826514
[6000]	training's mape: 0.00681398
[8000]	training's mape: 0.00596918
[10000]	training's mape: 0.00536002
[12000]	training's mape: 0.00488617
[14000]	training's mape: 0.00449979
[16000]	training's mape: 0.00417601
[18000]	training's mape: 0.00389735
[20000]	training's mape: 0.00365297
[22000]	training's mape: 0.00343615
[24000]	training's mape: 0.00324172
[26000]	training's mape: 0.00306515
[28000]	training's mape: 0.00290394
[30000]	training's mape: 0.00275664
[32000]	training's mape: 0.00262226
[34000]	training's mape: 0.0024977
[36000]	training's mape: 0.00238198
[38000]	training's mape: 0.00227376
[40000]	training's mape: 0.00217395
[42000]	training's mape: 0.00208054
[44000]	training's mape: 0.00199383
[46000]	training's mape: 0.00191195
[48000]	training's mape: 0.00183495
[50000]	training's mape: 0.00176216
[52000]	training's mape: 0.00169433
[54000]	training's mape: 0.00162997
[56000]	training's mape: 0.0015697

KeyboardInterrupt: 

In [ ]:
df_test_pred.to_csv('output/'+model_name+'-test-one.csv', index=False)

In [ ]:
print('Total run time:', t11 - t10)

### Feature importance

In [ ]:
lgb.plot_importance(model, figsize=(12, 30), height=0.5) # 050, 051, 052
plt.show()

In [ ]:
lgb.plot_importance(model, figsize=(12, 30), height=0.5, importance_type='gain') # 050, 051, 052
plt.show()